In [116]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

In [117]:
park = Customer('Park', 100)
park

Customer(name='Park', fidelity=100)

In [118]:
class LineItem:
    """구매할 물품/갯수 생성해서 총 가격을 반환"""
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

In [119]:
class Order: # Context
    """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion #할인 객체
    def total(self):
        """_total 속성이 없으면 전체 계산할 값을 계산"""
        if not hasattr(self, '_total'):
            self._total = sum(item.total() for item in self.cart)
        return self._total
    def due(self):
        """할인금액 차감"""
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self) # self = Order 객체
        return self.total() - discount
    def _repr_(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [120]:
class Promotion (ABC): # strategy: abstract base class
    """할인 혜택 클래스들의 형태를 선언"""

    @abstractmethod # 이 클래스를 상속하는 클래스는 반드시 이 메서드를 선언해야함
    def discount (self, order):
        """할인액을 구체적인 숫자로 반환"""
        pass

In [121]:
class FidelityPromo (Promotion):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    def discount (self, order): return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0
class BulkItemPromo (Promotion):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    def discount (self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount

class LargeOrderPromo (Promotion):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

In [122]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart= [LineItem('banana', 4, .5), LineItem('apple', 10, 1.5), LineItem('watermellon', 5, 5.0)]

In [123]:
Order(joe, cart, FidelityPromo())

In [124]:
Order(ann, cart, FidelityPromo())

In [125]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

In [126]:
Order(joe, banana_cart, BulkItemPromo())

In [127]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

In [128]:
Order(joe, long_order, LargeOrderPromo())

In [129]:
Order(joe, cart, LargeOrderPromo())

In [130]:
class Order: # Context
    """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion # 할인 객체
    def total(self):
        if not hasattr(self, '_total'):
            self._total = sum(item.total() for item in self.cart)
        return self._total
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion (self) # promotion은 함수 객체
        return self.total() - discount

    def _repr_(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [131]:
def fidelity_promo (order):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0
def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount
def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

In [132]:
Order(joe, long_order, fidelity_promo)

In [133]:
Order(ann, long_order, fidelity_promo)

In [134]:
Order(joe, banana_cart, bulk_item_promo)

In [135]:
Order(ann, long_order, large_order_promo)

In [136]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]
#함수들로 구현된 전략들의 리스트

def best_promo(order):
    """최대로 할인받을 금액을 반환"""
    return max(promo(order) for promo in promos)

In [137]:
Order(joe, long_order, best_promo)

In [138]:
Order(joe, banana_cart, best_promo)

In [139]:
Order(ann, cart, best_promo)

In [140]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', '_i2', 'ABC', 'abstractmethod', 'namedtuple', 'Customer', '_i3', '_i4', 'park', '_4', '_i5', '_i6', 'LineItem', '_i7', 'Order', '_i8', 'Promotion', '_i9', '_i11', '_i12', 'FidelityPromo', 'BulkItemPromo', '_i13', 'LargeOrderPromo', '_i14', 'joe', 'ann', '_i15', '_i16', '_i17', '_i18', '_i19', '_i20', '_i21', '_21', '_i22', '_i23', '_i24', '_i25', '_i26', '_i27', '_i28', '_i29', '_29', '_i30', '_i31', '_i32', '_i33', '_i34', '_i35', '_i36', '_i37', '_i38', '_i39', '_i40', '_i41', '_41', '_i42', '_i43', '_i44', '_i45', '_i46', '_i47', '_i48', '_48', '_i49', '_i50', '_i51', '_i52', '_i53', 'cart', '_i54', '_i55', '_i56', '_56', '_i57', '_i58', '_i59', '_i60', '_i61', '_i62', '_62', '_i63', '_63', '_i64', 'banana_cart', '_i65', '_65', '_i66', 'long_order', '_i67', '_67', '_i68

In [141]:
globals()['best_promo']

<function __main__.best_promo(order)>

In [142]:
promos = [globals()[name] for name in globals()
        if name.endswith('_promo')
        and name != 'best_promo']

promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

In [143]:
#07
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [144]:
@deco
def target():
    print('running target()')

target() # 실제로는 inner()를 실행

running inner()


In [145]:
target # inner()를 가리킴

<function __main__.deco.<locals>.inner()>

In [146]:
registry = [] # 여기에 decorated function 추가

def register(func):
    print('running register (%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
@register
def f2():
    print('running f2()')
def f3():
    print('running f3()')
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
if __name__=='__main__':
    main()

running register (<function f1 at 0x7ebfea9fdc60>)
running register (<function f2 at 0x7ec0144d2ef0>)
running main()
registry -> [<function f1 at 0x7ebfea9fdc60>, <function f2 at 0x7ec0144d2ef0>]
running f1()
running f2()
running f3()


In [147]:
promos = []
def promotion(promo_func):
    """promotion() decorator는 promo_func를 promos 리스트에 추가한 후 그대로 반환"""
    promos.append(promo_func)
    return promo_func
@promotion
def fidelity_promo (order):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0
@promotion
def bulk_item_promo (order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount
@promotion
def large_order_promo (order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0
def best_promo (order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo (order) for promo in promos)

In [148]:
def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

In [149]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [151]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
f3(3)

3
6


In [152]:
b

9

In [153]:
f3(3)

3
9


In [154]:
b=30
f3(3)

3
30


In [155]:
class Averager():
    def __init__(self):
        self.series = []
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [156]:
avg = Averager()
avg(10)

10.0

In [157]:
avg(11)

10.5

In [158]:
avg(12)

11.0

In [159]:
def make_averager():
    series = []
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager

In [160]:
avg = make_averager()
avg(10)

10.0

In [161]:
avg(11)

10.5

In [162]:
avg(12)

11.0

In [163]:
avg.__code__.co_varnames

('new_value', 'total')

In [164]:
avg.__code__.co_freevars

('series',)

In [165]:
avg.__closure__

(<cell at 0x7ebfeadf99f0: list object at 0x7ebfea8d95c0>,)

In [166]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [167]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    return averager

In [168]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [169]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager

In [170]:
avg = make_averager()
avg(10)

10.0

In [171]:
import time
def clock(func):
    def clocked (*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() -  t0
        name = func.__name__
        arg_str = ', '.join(repr (arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [172]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*'*40, 'Calling snooze(.123)')
    snooze(.123)
    print('*'*40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12318535s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000078s] factorial(1) -> 1
[0.00005852s] factorial(2) -> 2
[0.00009021s] factorial(3) -> 6
[0.00238261s] factorial(4) -> 24
[0.00245765s] factorial(5) -> 120
[0.00310554s] factorial(6) -> 720
6! = 720


In [179]:
import time
import functools
def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        arg_lst = []
        name = func. __name__
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k,w) for k,w in sorted (kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [180]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*'*40, 'Calling snooze(.123)')
    snooze(.123)
    print('*'*40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12316298s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000119s] factorial(1) -> 1
[0.00036454s] factorial(2) -> 2
[0.00072455s] factorial(3) -> 6
[0.00108218s] factorial(4) -> 24
[0.00142550s] factorial(5) -> 120
[0.00179863s] factorial(6) -> 720
6! = 720
